In [11]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import pandas_datareader.data as web

In [12]:
rawCovid = pd.read_csv("covid.csv")
variables = ["submission_date", "tot_cases", "new_case", "tot_death", "new_death"]
covid = rawCovid.loc[:,variables]

In [13]:
covid["submission_date"] = pd.to_datetime(covid["submission_date"])
covid.rename(columns={"submission_date":"Date"}, inplace=True)

In [14]:
weeklyCovid = covid.groupby(pd.Grouper(key = "Date", freq ="W")).sum()
weeklyCovid.reset_index(inplace=True)

weeklyCovid["case_change"] = weeklyCovid["tot_cases"].diff().fillna(0)/weeklyCovid["tot_cases"]
weeklyCovid["death_change"] = weeklyCovid["tot_death"].diff().fillna(0)/weeklyCovid["tot_death"]
weeklyCovid["death_change"].fillna(0, inplace=True)


In [25]:
bioStock = ["FBRX", "TRIL", "CODX", "ALIM", "BTAI"]
pharmStock = ["ABT", "RCUS", "LLY", "ELAN", "MRK"]
techStocks = ["BILI", "CCOI", "CHT"]
energyStocks = ["CVX","XOM","KMI","WMB","LNG"]
retailStocks = ["WMT","AMZN","COST","BIG","ACI"]
consumerStaplesStocks= ['VDC', 'VCSAX']
utilityStocks= [ 'VUIAX', 'VPU']
# please create lists of your stocks' ticker symbols


In [26]:
# please add your lists to tickerSymbol
tickerSymbol = bioStock + pharmStock + techStocks + energyStocks + retailStocks + consumerStaplesStocks + utilityStocks
# + "your lists"

In [27]:
start = "2020-01-22"
end = dt.now()

stocks = pd.DataFrame()
sc_df = pd.DataFrame()

for i in tickerSymbol:
    df = web.DataReader(i, "yahoo", start, end)
    df.reset_index(inplace=True)

    weeklyPrice = df.groupby([pd.Grouper(key="Date", freq="W")]).mean()
    weeklyPrice["Stock"] = i
    weeklyPrice.reset_index(inplace=True)
    
    stocks = stocks.append(weeklyPrice[["Adj Close", "Date", "Stock"]],
                          ignore_index = True)
    stocks["stock_price_change"] = stocks["Adj Close"].diff().fillna(0)
    
    sc_df = pd.merge(stocks, weeklyCovid, on="Date")

In [28]:
sc_df["Category"] = ""
sc_df.loc[sc_df["Stock"].isin(bioStock), "Category"] = "biotech"
sc_df.loc[sc_df["Stock"].isin(pharmStock), "Category"] = "pharmacy"
sc_df.loc[sc_df["Stock"].isin(bioStock), "Category"] = "it"
sc_df.loc[sc_df["Stock"].isin(pharmStock), "Category"] = "energy"
sc_df.loc[sc_df["Stock"].isin(pharmStock), "Category"] = "retail"
sc_df.loc[sc_df["Stock"].isin(bioStock), "Category"] = "consumer"
sc_df.loc[sc_df["Stock"].isin(pharmStock), "Category"] = "utility"

# please add your stocks' categories


In [29]:
sc_df["Rise/Fall"] = ""
sc_df.loc[sc_df["stock_price_change"]>0, "Rise/Fall"] = "Rise"
sc_df.loc[sc_df["stock_price_change"]<0, "Rise/Fall"] = "Fall"
sc_df.loc[sc_df["stock_price_change"]==0, "Rise/Fall"] = np.nan

sc_df.dropna(inplace=True)




In [30]:
sc_df

,Adj Close,Date,Stock,stock_price_change,tot_cases,new_case,tot_death,new_death,case_change,death_change,Category,Rise/Fall
1,3.996667,2020-01-26,TRIL,-31.423334,1,1.0,0,0.0,0.000000,0.000000,consumer,Fall
2,1.680000,2020-01-26,CODX,-11.462000,1,1.0,0,0.0,0.000000,0.000000,consumer,Fall
3,7.433333,2020-01-26,ALIM,-6.656667,1,1.0,0,0.0,0.000000,0.000000,consumer,Fall
4,17.196667,2020-01-26,BTAI,13.211267,1,1.0,0,0.0,0.000000,0.000000,consumer,Rise
5,89.964806,2020-01-26,ABT,38.262807,1,1.0,0,0.0,0.000000,0.000000,utility,Rise
...,...,...,...,...,...,...,...,...,...,...,...,...
1107,15.268000,2020-11-08,ACI,0.340000,1161657,10032.0,25108,53.0,-5.808244,-5.920225,,Rise
1108,164.888000,2020-11-08,VDC,2.834000,1161657,10032.0,25108,53.0,-5.808244,-5.920225,,Rise
1109,81.072502,2020-11-08,VCSAX,1.182503,1161657,10032.0,25108,53.0,-5.808244,-5.920225,,Rise
1110,69.767500,2020-11-08,VUIAX,0.677501,1161657,10032.0,25108,53.0,-5.808244,-5.920225,,Rise
